In [ ]:
#!pip install polygon-api-client

In [ ]:
#import system and source code folder
import sys
sys.path.append("../SourceCode")

In [ ]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#Imports 
#Pandas
import pandas as pd
#Numpy
import numpy as np
#Tqdm
from tqdm import tqdm
#Hvplots
import hvplot.pandas
#Matplotlib
import matplotlib.pyplot as plt
#Import OS
import os
#Json
import json
#TA library
import talib as talib
#Spicy
from scipy.stats import linregress

from pathlib import Path

In [ ]:
#datatime 
from datetime import date, datetime
#Typing 
from typing import Any, Optional
#HTTP Addaptor
from requests.adapters import HTTPAdapter
#urllib3 is a powerful, user-friendly HTTP client
from urllib3.util.retry import Retry
#Polygon 3rd Party API for Data and Restful Client
from polygon import RESTClient
#Polygon Key
from local_settings import polygon_api_key as settings

# Defining Functions 


In [ ]:
# Connectivity Function Validation

markets = ['crypto', 'stocks', 'fx']

class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str=settings['api_key'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)

In [ ]:
# Loading API KEY 
client = MyRESTClient(settings['api_key'])


In [ ]:
# Testing API Client 

help(client.reference_tickers_v3)


In [ ]:


class MyRESTClient(RESTClient):
    # Defining Function with Time Out after 5 intervales against error codes (429, 500, 502, 503, 504) and implementing retry mechensim.
    def __init__(self, auth_key: str=settings['api_key'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)

    # Defining function for getting market data
    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                # Only use USD pairings.</em>
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None

# Functions Test for Polygon API


In [ ]:
# Getting the list of Cypto Currencies. 

client = MyRESTClient(settings['api_key'])
df = client.get_tickers(market='crypto')
df

In [ ]:
# Getting Help for Stock, Equities and Crytos
help(client.stocks_equities_aggregates)

In [ ]:
# Defining Class to gather Currencies with Pologon API using retry mechinism and fetching minute bar data with Open, High, low amd Close. 
class MyRESTClient(RESTClient):
    # Defining Retry mechennism.
    def __init__(self, auth_key: str=settings['api_key'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)
   
    # Defining Tickers and markets. 
    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                # Only use USD pairings.</em>
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None
    # Definfing function for getting Minute bar data.
    def get_bars(self, market:str=None, ticker:str=None, multiplier:int=1,
                 timespan:str='minute', from_:date=None, to:date=None) -> pd.DataFrame:

        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        if ticker is None:
            raise Exception('Ticker must not be None.')

        from_ = from_ if from_ else date(2000,1,1)
        to = to if to else date.today()

        if market == 'crypto':
            resp = self.crypto_aggregates(ticker, multiplier, timespan,
                                          from_.strftime('%Y-%m-%d'), to.strftime('%Y-%m-%d'),
                                          limit=50000)
            df = pd.DataFrame(resp.results)
            last_minute = 0
            while resp.results[-1]['t'] > last_minute:
                last_minute = resp.results[-1]['t'] # Last minute in response</em>
                last_minute_date = datetime.fromtimestamp(last_minute/1000).strftime('%Y-%m-%d')
                resp = self.crypto_aggregates(ticker, multiplier, timespan,
                                          last_minute_date, to.strftime('%Y-%m-%d'),
                                          limit=50000)
                new_bars = pd.DataFrame(resp.results)
                df = df.append(new_bars[new_bars['t'] > last_minute])

            df['date'] = pd.to_datetime(df['t'], unit='ms')
            df = df.rename(columns={'o':'open',
                                    'h':'high',
                                    'l':'low',
                                    'c':'close',
                                    'v':'volume',
                                    'vw':'vwap',
                                    'n':'transactions'})
            df = df[['date','open','high','low','close','volume']]

            return df
        return None

In [ ]:
# Defining a Function for regression slope. 
def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    return slope


# Data Gathering for Cryptocurrencies 

## BTC Data for 2 years on minutes bars 

In [ ]:
start = datetime(2000,1,1)
client = MyRESTClient(settings['api_key'])
df = client.get_bars(market='crypto', ticker='X:BTCUSD', from_=start)
df

In [ ]:
BTC_df = pd.DataFrame()

BTC_df = df 

In [ ]:
BTC_df.hvplot(x='date', y=['close'],
                value_label='Price', subplots=True, width=600, height=400)

## ETH Data for 2 years on minutes bars


In [ ]:
start = datetime(2000,1,1)
client = MyRESTClient(settings['api_key'])
ETH_df = client.get_bars(market='crypto', ticker='X:ETHUSD', from_=start)
ETH_df

In [ ]:
ETH_df.hvplot(x='date', y=['close'],
                value_label='Price', subplots=True, width=600, height=400)

# BNB Data for 2 years on minutes bars

In [ ]:
start = datetime(2000,1,1)
client = MyRESTClient(settings['api_key'])
BNB_df = client.get_bars(market='crypto', ticker='X:BNBUSD', from_=start)
BNB_df

# XRP Data for 2 years on minutes bars


In [ ]:
start = datetime(2000,1,1)
client = MyRESTClient(settings['api_key'])
XRP_df = client.get_bars(market='crypto', ticker='X:XRPUSD', from_=start)
XRP_df

# Doge Data for 2 years on minutes bars

In [ ]:
start = datetime(2000,1,1)
client = MyRESTClient(settings['api_key'])
DOGE_df = client.get_bars(market='crypto', ticker='X:DOGEUSD', from_=start)
DOGE_df

## ADA Data for 2 years on minutes bars

In [ ]:
start = datetime(2000,1,1)
client = MyRESTClient(settings['api_key'])
ADA_df = client.get_bars(market='crypto', ticker='X:ADAUSD', from_=start)
ADA_df

## Matic Data for 2 years on minutes bars


In [ ]:
start = datetime(2000,1,1)
client = MyRESTClient(settings['api_key'])
MATIC_df = client.get_bars(market='crypto', ticker='X:MATICUSD', from_=start)
MATIC_df

# INDICATORS 

## Name added to dataframe 
## close percentage 
## rolling slope over 30 bars 
## Range
## Range mean over 30 bars 
## Range Standard Divation over 30 bars
## Range Z-Score
## Y Shifted over 1 minutes
## SMA Fast over 30 bars over close 
## SMA Slow over 100 bars over close 
## RSI 
## Volume percentage 
## SMA Slow Percentage Change over 30 mins 
## SMA Fast Percentage Change over 100 mins
## Rolling Mean
## Rolling STD
## Bollinger Band High over close 
## Bollinger Bank Low over close 


In [ ]:
short_window =30
long_window = 100
minutes_back = 30

BTC_df.insert(1, 'CRYPTO', 'BTC')
BTC_df['ClosePct']=(BTC_df['close']-BTC_df['low'])/(BTC_df['high']-BTC_df['low'])
BTC_df['rolling_slope'] = BTC_df['close'].rolling(window=minutes_back).apply(get_slope, raw=False)
BTC_df['Range']=BTC_df['high']-BTC_df['low']
BTC_df['RangeMa']=BTC_df['Range'].rolling(30).mean()
BTC_df['RangeSD']=BTC_df['Range'].rolling(30).std()
BTC_df['RangeZscore']=(BTC_df['Range']-BTC_df['RangeMa'])/df['RangeSD']
BTC_df['Y']=BTC_df['close'].pct_change().shift(-1)
BTC_df['SMA_Fast'] = BTC_df['close'].rolling(window=short_window).mean()/BTC_df['close']
BTC_df['SMA_Slow'] = BTC_df['close'].rolling(window=long_window).mean()/BTC_df['close']
BTC_df['RSI']=talib.RSI(BTC_df['close'], timeperiod=30)
BTC_df['Volume_PCT'] = BTC_df['volume'].pct_change()
BTC_df['SMA_Fast_'] = BTC_df['close'].rolling(window=short_window).mean()
BTC_df['SMA_Slow_'] = BTC_df['close'].rolling(window=long_window).mean()
BTC_df['SMA_Fast_pctchange']=BTC_df['SMA_Fast_'].pct_change()
BTC_df['SMA_Slow_pctchange']=BTC_df['SMA_Slow_'].pct_change()
BTC_df['rolling_mean'] = BTC_df['close'].rolling(short_window).mean()
BTC_df['rolling_std'] = BTC_df['close'].rolling(short_window).std()
BTC_df['Bollinger High'] = (BTC_df['rolling_mean'] + (BTC_df['rolling_std'] * 2))/BTC_df['close']
BTC_df['Bollinger Low'] = (BTC_df['rolling_mean'] - (BTC_df['rolling_std'] * 2))/BTC_df['close']

BTC_df

In [ ]:
BTC_df['Y']=BTC_df['close'].pct_change().shift(-1)

In [ ]:
ETH_df.insert(1, 'CRYPTO', 'ETH')

#short_window =30
#long_window = 720
#minutes_back=30

ETH_df['ClosePct']=(ETH_df['close']-ETH_df['low'])/(ETH_df['high']-ETH_df['low'])
ETH_df['Range']=ETH_df['high']-ETH_df['low']
ETH_df['RangeMa']=ETH_df['Range'].rolling(30).mean()
ETH_df['RangeSD']=ETH_df['Range'].rolling(30).std()
ETH_df['RangeZscore']=(ETH_df['Range']-ETH_df['RangeMa'])/ETH_df['RangeSD']
ETH_df['SMA_Fast'] = ETH_df['close'].rolling(window=short_window).mean()/ETH_df['close']
ETH_df['SMA_Slow'] = ETH_df['close'].rolling(window=long_window).mean()/ETH_df['close']
ETH_df['RSI']=talib.RSI(ETH_df['close'], timeperiod=30)
ETH_df['Volume_PCT'] = ETH_df['volume'].pct_change()
ETH_df['SMA_Fast_'] = ETH_df['close'].rolling(window=short_window).mean()
ETH_df['SMA_Slow_'] = ETH_df['close'].rolling(window=long_window).mean()
ETH_df['SMA_Fast_pctchange']=ETH_df['SMA_Fast_'].pct_change()
ETH_df['SMA_Slow_pctchange']=ETH_df['SMA_Slow_'].pct_change()
ETH_df['rolling_mean'] = ETH_df['close'].rolling(short_window).mean()
ETH_df['rolling_std'] = ETH_df['close'].rolling(short_window).std()
ETH_df['Bollinger High'] = (ETH_df['rolling_mean'] + (ETH_df['rolling_std'] * 2))/ETH_df['close']
ETH_df['Bollinger Low'] = (ETH_df['rolling_mean'] - (ETH_df['rolling_std'] * 2))/ETH_df['close']
ETH_df['rolling_slope'] = ETH_df['close'].rolling(window=minutes_back).apply(get_slope, raw=False)
ETH_df['Y']=ETH_df['close'].pct_change().shift(-1)


ETH_df

In [ ]:
BNB_df.insert(1, 'CRYPTO', 'BNB')

#short_window =30
#long_window = 720
#minutes_back=30

BNB_df['ClosePct']=(BNB_df['close']-BNB_df['low'])/(BNB_df['high']-BNB_df['low'])
BNB_df['Range']=BNB_df['high']-BNB_df['low']
BNB_df['RangeMa']=BNB_df['Range'].rolling(30).mean()
BNB_df['RangeSD']=BNB_df['Range'].rolling(30).std()
BNB_df['RangeZscore']=(BNB_df['Range']-BNB_df['RangeMa'])/BNB_df['RangeSD']
BNB_df['SMA_Fast'] = BNB_df['close'].rolling(window=short_window).mean()/BNB_df['close']
BNB_df['SMA_Slow'] = BNB_df['close'].rolling(window=long_window).mean()/BNB_df['close']
BNB_df['RSI']=talib.RSI(BNB_df['close'], timeperiod=30)
BNB_df['Volume_PCT'] = BNB_df['volume'].pct_change()
BNB_df['SMA_Fast_'] = BNB_df['close'].rolling(window=short_window).mean()
BNB_df['SMA_Slow_'] = BNB_df['close'].rolling(window=long_window).mean()
BNB_df['SMA_Fast_pctchange']=BNB_df['SMA_Fast_'].pct_change()
BNB_df['SMA_Slow_pctchange']=BNB_df['SMA_Slow_'].pct_change()
BNB_df['rolling_mean'] = BNB_df['close'].rolling(short_window).mean()
BNB_df['rolling_std'] = BNB_df['close'].rolling(short_window).std()
BNB_df['Bollinger High'] = (BNB_df['rolling_mean'] + (BNB_df['rolling_std'] * 2))/BNB_df['close']
BNB_df['Bollinger Low'] = (BNB_df['rolling_mean'] - (BNB_df['rolling_std'] * 2))/BNB_df['close']
BNB_df['rolling_slope'] = BNB_df['close'].rolling(window=minutes_back).apply(get_slope, raw=False)


BNB_df['Y']=BNB_df['close'].pct_change().shift(-1)


# Generate the fast and slow simple moving averages 30mins and 12 hous, respectively with respect to current day close

BNB_df

In [ ]:
XRP_df.insert(1, 'CRYPTO', 'XRP')

#short_window =30
#long_window = 720
#minutes_back=30

XRP_df['ClosePct']=(XRP_df['close']-XRP_df['low'])/(XRP_df['high']-XRP_df['low'])
XRP_df['Range']=XRP_df['high']-XRP_df['low']
XRP_df['RangeMa']=XRP_df['Range'].rolling(30).mean()
XRP_df['RangeSD']=XRP_df['Range'].rolling(30).std()
XRP_df['RangeZscore']=(XRP_df['Range']-XRP_df['RangeMa'])/XRP_df['RangeSD']
XRP_df['SMA_Fast'] = XRP_df['close'].rolling(window=short_window).mean()/XRP_df['close']
XRP_df['SMA_Slow'] = XRP_df['close'].rolling(window=long_window).mean()/XRP_df['close']
XRP_df['RSI']=talib.RSI(XRP_df['close'], timeperiod=30)
XRP_df['Volume_PCT'] = XRP_df['volume'].pct_change()
XRP_df['SMA_Fast_'] = XRP_df['close'].rolling(window=short_window).mean()
XRP_df['SMA_Slow_'] = XRP_df['close'].rolling(window=long_window).mean()
XRP_df['SMA_Fast_pctchange']=XRP_df['SMA_Fast_'].pct_change()
XRP_df['SMA_Slow_pctchange']=XRP_df['SMA_Slow_'].pct_change()
XRP_df['rolling_mean'] = XRP_df['close'].rolling(short_window).mean()
XRP_df['rolling_std'] = XRP_df['close'].rolling(short_window).std()
XRP_df['Bollinger High'] = (XRP_df['rolling_mean'] + (XRP_df['rolling_std'] * 2))/XRP_df['close']
XRP_df['Bollinger Low'] = (XRP_df['rolling_mean'] - (XRP_df['rolling_std'] * 2))/XRP_df['close']
XRP_df['rolling_slope'] = XRP_df['close'].rolling(window=minutes_back).apply(get_slope, raw=False)


XRP_df['Y']=XRP_df['close'].pct_change().shift(-1)


XRP_df

In [ ]:
#DOGE_df.insert(1, 'CRYPTO', 'DOGE')

#short_window =30
#long_window = 720
#minutes_back=30

DOGE_df['ClosePct']=(DOGE_df['close']-DOGE_df['low'])/(DOGE_df['high']-DOGE_df['low'])
DOGE_df['Range']=DOGE_df['high']-DOGE_df['low']
DOGE_df['RangeMa']=DOGE_df['Range'].rolling(30).mean()
DOGE_df['RangeSD']=DOGE_df['Range'].rolling(30).std()
DOGE_df['RangeZscore']=(DOGE_df['Range']-DOGE_df['RangeMa'])/DOGE_df['RangeSD']
DOGE_df['SMA_Fast'] = DOGE_df['close'].rolling(window=short_window).mean()/DOGE_df['close']
DOGE_df['SMA_Slow'] = DOGE_df['close'].rolling(window=long_window).mean()/DOGE_df['close']
DOGE_df['RSI']=talib.RSI(DOGE_df['close'], timeperiod=30)
DOGE_df['Volume_PCT'] = DOGE_df['volume'].pct_change()
DOGE_df['SMA_Fast_'] = DOGE_df['close'].rolling(window=short_window).mean()
DOGE_df['SMA_Slow_'] = DOGE_df['close'].rolling(window=long_window).mean()
DOGE_df['SMA_Fast_pctchange']=DOGE_df['SMA_Fast_'].pct_change()
DOGE_df['SMA_Slow_pctchange']=DOGE_df['SMA_Slow_'].pct_change()
DOGE_df['rolling_mean'] = DOGE_df['close'].rolling(short_window).mean()
DOGE_df['rolling_std'] = DOGE_df['close'].rolling(short_window).std()
DOGE_df['Bollinger High'] = (DOGE_df['rolling_mean'] + (DOGE_df['rolling_std'] * 2))/DOGE_df['close']
DOGE_df['Bollinger Low'] = (DOGE_df['rolling_mean'] - (DOGE_df['rolling_std'] * 2))/DOGE_df['close']
DOGE_df['rolling_slope'] = DOGE_df['close'].rolling(window=minutes_back).apply(get_slope, raw=False)


DOGE_df['Y']=DOGE_df['close'].pct_change().shift(-1)


DOGE_df

In [ ]:
MATIC_df.insert(1, 'CRYPTO', 'MATIC')
#short_window =30
#long_window = 720
#minutes_back=30

MATIC_df['ClosePct']=(MATIC_df['close']-MATIC_df['low'])/(MATIC_df['high']-MATIC_df['low'])
MATIC_df['Range']=MATIC_df['high']-MATIC_df['low']
MATIC_df['RangeMa']=MATIC_df['Range'].rolling(30).mean()
MATIC_df['RangeSD']=MATIC_df['Range'].rolling(30).std()
MATIC_df['RangeZscore']=(MATIC_df['Range']-MATIC_df['RangeMa'])/MATIC_df['RangeSD']
MATIC_df['SMA_Fast'] = MATIC_df['close'].rolling(window=short_window).mean()/MATIC_df['close']
MATIC_df['SMA_Slow'] =MATIC_df['close'].rolling(window=long_window).mean()/MATIC_df['close']
MATIC_df['RSI']=talib.RSI(MATIC_df['close'], timeperiod=30)
MATIC_df['Volume_PCT'] = MATIC_df['volume'].pct_change()
MATIC_df['SMA_Fast_'] = MATIC_df['close'].rolling(window=short_window).mean()
MATIC_df['SMA_Slow_'] = MATIC_df['close'].rolling(window=long_window).mean()
MATIC_df['SMA_Fast_pctchange']=MATIC_df['SMA_Fast_'].pct_change()
MATIC_df['SMA_Slow_pctchange']=MATIC_df['SMA_Slow_'].pct_change()
MATIC_df['rolling_mean'] = MATIC_df['close'].rolling(short_window).mean()
MATIC_df['rolling_std'] = MATIC_df['close'].rolling(short_window).std()
MATIC_df['Bollinger High'] = (MATIC_df['rolling_mean'] + (MATIC_df['rolling_std'] * 2))/MATIC_df['close']
MATIC_df['Bollinger Low'] = (MATIC_df['rolling_mean'] - (MATIC_df['rolling_std'] * 2))/MATIC_df['close']
MATIC_df['rolling_slope'] = MATIC_df['close'].rolling(window=minutes_back).apply(get_slope, raw=False)


MATIC_df['Y']=MATIC_df['close'].pct_change().shift(-1)


MATIC_df

# Combined Dataframe for Predictive Models

# Droping Columns from Combined Data Frame

In [ ]:
# Create empty dataframe to hold stock data
df_all_crypto=pd.DataFrame()
#create a list of data frames
list = [BTC_df,ETH_df,BNB_df,XRP_df,DOGE_df,MATIC_df,ADA_df]

# Combine individual stocks into a single data frame
df_all_crypto=pd.concat(list,axis=0)
# Drop the N/As
df_all_crypto = df_all_crypto.dropna()
df_all_crypto=df_all_crypto.sort_values(['date','CRYPTO']).set_index(['date','CRYPTO'])


In [ ]:
df_all_crypto.iloc[-1]

In [ ]:
df_all_crypto.iloc[4000000]

In [ ]:
df_all_crypto.drop(columns = ['open','high','low','close','volume','Range','RangeMa'], inplace = True)


In [ ]:
df_all_crypto 

## Shifting Y to 15 mins


In [ ]:
df_all_crypto.loc[:, 'Y'] = df_all_crypto.Y.shift(-14)


In [ ]:
# Dropinging NAN and NA

df_all_crypto = df_all_crypto.dropna()

In [ ]:
df_all_crypto

In [ ]:
BTC_df

# Algorithm Trading & Analysis 

In [ ]:
signals_df = pd.DataFrame()
signals_df = ETH_df[['date','close','ClosePct','SMA_Fast','SMA_Slow','RSI','Bollinger High','Bollinger Low','rolling_slope']].copy()
signals_df = signals_df.set_index(['date'])
signals_df = signals_df.dropna()


In [ ]:
# Prepopulate the `Signal` for trading
signals_df['Signal'] = 0.0
RSI_Buy = 70
RSI_Sell = 30
Slope_Buy = 0.1
Slope_sell= -0.1
# Generate the trading signal 1 or -1,
# where 1 is when short-window (SMA50) is greater than the long (SMA 100)
# and 0 otherwise

for index, row in signals_df.iterrows():
    if row["ClosePct"] > row["Bollinger Low"] and row["SMA_Fast"] > row["SMA_Slow"] and row["RSI"] >= RSI_Buy and row['rolling_slope'] > row["SMA_Fast"] :
        signals_df.loc[index, "Signal"] = -1.0
    if  row["ClosePct"] < row["Bollinger High"]and row["SMA_Fast"] < row["SMA_Slow"] and RSI_Sell >= row["RSI"] and row['rolling_slope'] < row["SMA_Fast"]:
        signals_df.loc[index,"Signal"] = 1.0

signals_df.head(10)





In [ ]:
signals_df['rolling_slope'].max()

In [ ]:
signals_df

In [ ]:
signals_df['Signal'].value_counts()

In [ ]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Signal"] == 1.0]["close"].hvplot.scatter(
    color="green",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Signal"] == -1.0]["close"].hvplot.scatter(
    color="red",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)    
strategy_plot = security_close * entry * exit
strategy_plot


In [ ]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 50

signals_df['Position'] = share_size * signals_df['Signal']
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum() 
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
signals_df.tail(10)


In [ ]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Signal'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Signal'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title=" Algorithm - Total Portfolio Value",
    yformatter='%.0f')

# Set Up the Portfolio-Level Risk/Reward Evaluation Metrics

In [ ]:
# Create a list for the column name
columns = ["Backtest"]

# Create a list holding the names of the new evaluation metrics
metrics = [
    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio",
    "Sortino Ratio"]

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

In [ ]:
signals_df.iloc[-1,-1]

In [ ]:
# Calculate annualized return
portfolio_evaluation_df.loc["Annualized Return"] = (
    signals_df["Portfolio Daily Returns"].mean() * 31536000
)

# Calculate cumulative return
portfolio_evaluation_df.loc["Cumulative Returns"] = signals_df.iloc[-1,-1]


# Calculate annual volatility
portfolio_evaluation_df.loc["Annual Volatility"] = (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(31536000)
)

# Review the result
portfolio_evaluation_df

# Calculate Sharpe ratio
portfolio_evaluation_df.loc["Sharpe Ratio"] = (
    signals_df["Portfolio Daily Returns"].mean() * 31536000) / (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(31536000)
)

# Review the result
portfolio_evaluation_df

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[["Portfolio Daily Returns"]].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,"Downside Returns"] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df["Portfolio Daily Returns"] < 0,
                     "Downside Returns"] = sortino_ratio_df["Portfolio Daily Returns"]**2

# Review the DataFrame
sortino_ratio_df.tail()

# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df["Portfolio Daily Returns"].mean() * 252
)

# Print the result
print(f"Annualized Return: {annualized_return}")

# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df["Downside Returns"].mean()) * np.sqrt(252)
)

# Print the result
print(f"Annualized Downside STD: {downside_standard_deviation}")

#The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc["Sortino Ratio"] = sortino_ratio

# Review the DataFrame
portfolio_evaluation_df

In [ ]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        "Stock",
        "Entry Date",
        "Exit Date",
        "Shares",
        "Entry Share Price",
        "Exit Share Price",
        "Entry Portfolio Holding",
        "Exit Portfolio Holding",
        "Profit/Loss"]
)
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row["Signal"] == 1:
        entry_date = index
        entry_portfolio_holding = row["Portfolio Holdings"]
        share_size = row["Entry/Exit Position"]
        entry_share_price = row["close"]

    elif row["Signal"] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row["close"] * row["Entry/Exit Position"])
        exit_share_price = row["close"]
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                "Stock": "ETH",
                "Entry Date": entry_date,
                "Exit Date": exit_date,
                "Shares": share_size,
                "Entry Share Price": entry_share_price,
                "Exit Share Price": exit_share_price,
                "Entry Portfolio Holding": entry_portfolio_holding,
                "Exit Portfolio Holding": exit_portfolio_holding,
                "Profit/Loss": profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

In [ ]:
trade_evaluation_df['Profit/Loss'].cumsum().hvplot()

# AI based prediction on pricing 


## Logitic Regression for predicting Buy and sell Signals 


In [ ]:
df_all_crypto

In [ ]:
#df_all_crypto['Signal']=df_all_crypto['Signal'].shift(-15)
df_all_crypto = df_all_crypto.dropna()

In [ ]:
df_all_crypto.tail(15)

In [ ]:
df_all_crypto['Signal'] = 0.0
for index, row in df_all_crypto.iterrows():
    if row["ClosePct"] < row["Bollinger High"]and row["SMA_Fast"] < row["SMA_Slow"] and RSI_Sell >= row["RSI"] and row['rolling_slope'] < row["SMA_Fast"]:
        df_all_crypto.loc[index, "Signal"] = 1.0
    elif row["ClosePct"] > row["Bollinger Low"] and row["SMA_Fast"] > row["SMA_Slow"] and row["RSI"] >= RSI_Buy and row['rolling_slope'] > row["SMA_Fast"] :
        df_all_crypto.loc[index, "Signal"] = -1.0
    else :
        df_all_crypto.loc[index, "Signal"] = -1.0

In [ ]:
df_all_crypto.dropna()

In [ ]:
#Covert data into features and detination parameter

X = df_all_crypto.drop(['RangeSD','SMA_Fast_','SMA_Slow_','rolling_mean','rolling_mean','Signal', 'Y'],axis=1).shift(-15).dropna()


In [ ]:
y_Signal = df_all_crypto['Signal']

In [ ]:
y_Signal.value_counts()

In [ ]:
y = df_all_crypto['Y']

In [ ]:
X_train=X.iloc[0:100000]
X_test=X.iloc[-5000:]
y_train=y_Signal.iloc[0:100000]
y_test=y_Signal.iloc[-5000:]
print(X_train)
print(y_train)
print(X_test)
print(y_test)
y_train.value_counts()

In [ ]:
# Import the LogisticRegression and Standard Scaler module from SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Scale the data
scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1, max_iter= 3000)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train_scaled, y_train)

In [ ]:
# Make a prediction using the testing data
testing_predictions_lr = lr_model.predict(X_test_scaled)

# Review the predictions
testing_predictions_lr


In [ ]:

np.argwhere(np.isnan(testing_predictions_lr))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy_score(y_test, testing_predictions_lr)

In [ ]:
confusion_matrix(y_test, testing_predictions_lr)

In [ ]:
classification_report(y_test, testing_predictions_lr)

# Neural Network Prodictions 


In [ ]:
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Activation, Dropout
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping


In [ ]:
# Define the model - deep neural net with two hidden layers
number_input_features = 12
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 4

# Create a sequential neural network model
nn_1 = Sequential()

# Add the first hidden layer
nn_1.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer
nn_1.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer
nn_1.add(Dense(units=1, activation="linear"))

In [ ]:
y_test=y.iloc[-5000:]

In [ ]:
nn_1.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [ ]:
# Fit the model
deep_net_model_1 = nn_1.fit(X_train_scaled, y_train, epochs=10)

In [ ]:
# Set the model's file path
file_path = Path("SourceCode/TraingModel1.h5")

# Export your model to an HDF5 file
nn_1.save(file_path)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled, y_test, verbose=2)

# Display evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
import gc
gc.collect()

In [ ]:
ls -lh
